# 이상적인 반복
- 파이썬에서 반복을 할 때 유용하게 사용할 수 있는 관용적인 코드  
- 제너레이터 활용

## 관용적인 반복 코드
내장함수인 enumerate()는 이터러블을 입력받아 인덱스 번호와 원본의 원소를 튜플형태로 변환하여 enumerate 객체를 반환함

In [1]:
list(enumerate("abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

더 저수준에서 enumerate와 유사한 객체를 생성  
이 객체는 단순히 시작 값을 입력하면 무한 시퀀스를 만드는 역할을 함  
객체의 next 함수를 호출할 때마다 다음 시퀀스 값을 무제한 출력해줌

In [2]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start
    
    def next(self):
        current = self.current
        self.current += 1
        return current

이 인터페이스에 기반을 두어 클라이언트를 작성하면 명시적으로 next() 함수를 호출해야 함

In [3]:
seq = NumberSequence()
print(seq.next())
print(seq.next())

seq2 = NumberSequence(10)
print(seq2.next())
print(seq2.next())

0
1
10
11


이 코드로 enumerate() 함수를 사용하도록 재작성할 수는 없음  
이는 일반 파이썬의 for 루프를 사용하기 위한 인터페이스를 지원하지 않기 때문  
따라서 이터러블 형태의 파라미터로도 사용할 수 없음

In [4]:
list(zip(NumberSequence(), "abcedf"))

TypeError: 'NumberSequence' object is not iterable

위처럼 iterable 하지 않아 이터러블 형태의 파라미터로 사용할 수 없음을 확인할 수 있음  

NumberSequence가 반복을 지원하지 않기 때문에 이와 같은 문제가 생기므로 이 문제를 해결하기 위해 &#95;&#95;iter&#95;&#95;() 매직 메서드를 구현하여 객체가 반복 가능하게 만들어야 함  
또한 next() 메서드를 수정하여 &#95;&#95;next&#95;&#95; 매직 메서드를 구현하면 객체는 이터레이터가 됨

In [5]:
class SequenceOfNumbers:
    
    def __init__(self, start=0):
        self.current = start
        
    def __next__(self):
        current = self.current
        self.current += 1
        return current
    
    def __iter__(self):
        return self

이렇게 하면 요소를 반복할 수 있을 뿐만 아니라, next() 메서드를 호출할 필요도 없음  
-> &#95;&#95;next&#95;&#95;() 메서드를 구현했으므로 next() 내장함수를 사용할 수 있기 때문

In [6]:
list(zip(SequenceOfNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [7]:
seq = SequenceOfNumbers(100)
print(next(seq))
print(next(seq))

100
101


### next() 함수
next() 내장함수는 이터러블을 다음 요소로 이동시키고 기존의 값을 반환함  
만약 이터레이터가 더 이상의 값을 가지고 있지 않다면 **StopIteration** 예외 발생  
**StopIteration** 예외는 반복이 끝났음을 나타내며 사용할 수 있는 요소가 더 이상 없음을 나타냄  

In [9]:
word = iter("hello")
print(next(word))
print(next(word))
print(next(word))
print(next(word))
print(next(word))
print(next(word))

h
e
l
l
o


StopIteration: 

StopIteration이 발생하는 문제를 해결하고 싶다면, StopIteration 예외를 캐치하는 것 외에도 next() 함수의 두번째 파라미터에 기본 값을 제공할 수도 있음  
기본값을 제공하면 StopIteration을 발생시키는 것 대신 기본값을 반환함

In [10]:
next(word, "default_value")

'default_value'

### 제너레이터 사용하기
앞의 코드는 제너레이터를 사용하여 훨씬 간단하게 작성할 수 있음  
제너레이터를 사용하려면 클래스 대신 필요한 값을 yield 하는 함수를 만들면 됨

In [11]:
def sequence(start=0):
    while True:
        yield start
        start += 1

**yield** 키워드가 해당 함수를 제너레이터로 만들어 줌  
위의 함수는 제너레이터이기 때문에 무한 루프를 사용해도 안전함  
제너레이터 함수가 호출되면 yield 문장을 만나기 전까지 실행되고 값을 생성한 후 그 자리에서 멈춤

In [12]:
seq = sequence(10)
print(next(seq))
print(next(seq))
list(zip(sequence(), "abcdef"))

10
11


[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

### Itertools
이터레이션이 파이썬의 중요한 컴포넌트이기 때문에 이터러블로 작업하면 코드가 파이썬과 더 어울린다는 장점이 있음  
itertools 모듈을 사용하면 이터러블의 기능을 온전히 활용할 수 있음  
위에서 사용한 sequence() 제너레이터는 itertoosl.count()와 유사  

이터레이터, 제너레이터, itertools를 연결하여 새로운 객체를 만드는 것도 가능함  
앞 예제에서 구매 이력 지표를 계산할 때 특정 기준을 넘은 값에 대해서만 연산을 하는 기능을 넣기위해 가장 간단한 방법은 반복문 내에 조건을 추가하는 것

아래의 process() 함수에 조건이 추가됨

In [25]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()
        
    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")
    
    def process(self):
        """
        반복문 내에 조건문을 아래와 같이 추가
        """
        for purchase_value in self.purchases:
            if purchase_value > 1000.0:
                print(f"too many items: {purchase_value}")
                continue
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self
    
    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value
            
    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value
            
    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1
        
    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )

이와 같은 코드는 파이썬스럽지 못하며 지나치게 엄격함.  
또한 위와 같은 코드는 기준 수치가 변경된다면 어떻게 해당 수치를 전달할지, 파라미터가 둘 이상 필요하면 어떻게 할지 등 고려할 사항이 많음  
위의 객체의 책임은 구매 이력에 대하여 잘 정의된 지표 값을 계산하고 출력하는 것이므로 다른 요구사항을 위의 객체에 반영하는 것은 부적절함.  

객체의 책임을 명확하게 하고 클라이언트의 요구로부터 독립시키기 위해 코드를 수정하는 대신 유지하고 클라잉언트 클래스의 요구사항이 무엇이든 그에 맞게 필터링하여 새로운 데이터를 만든다고 가정  
1000개 넘게 구매한 이력의 처음 10개만 처리하려고 하면 다음과 같이 하면 됨

In [36]:
from itertools import islice

purchases = [1, 2, 3, 4, 5, 1001, 6, 7, 8, 9, 10, 11]
purchases = islice(filter(lambda p: p > 1000, purchases), 10)
"""
for purchase in list(purchases):
    print(purchase)
하면 1001이 출력되어 purchases 내에 1001이 있음을 확인 가능
"""
stats = PurchasesStats(purchases).process()

이런 식으로 필터링을 해도 메모리의 손해는 없음  
이는 모든 것이 제너레이터이므로 전체에서 필터링해 연산한 것처럼 보이지만 실제로는 하나씩 가져오므로 모든 것을 메모리에 올릴 필요가 없기 때문

#### 이터레이터를 사용한 코드 간소화  
이터레이터에 대해 살펴보았고 itertools 모듈까지 확인했으므로 앞의 예제를 간소화 할 수 있음

In [37]:
def process_purchases(purchases):
    min_, max_, avg = itertools.tee(purchases, 3)
    return min(min_), max(max_), median(avg)

이 예제에서 itertools.tee는 원래의 이터러블을 세 개의 새로운 이터러블로 분할하여 구매이력을 세 번 반복할 필요 없이 분할된 이터러블을 사용해 필요한 연산을 함  
purchases 파라미터에 다른 이터러블 객체를 넘기면 itertoosl.tee 함수 덕분에 오직 한 번만 순회하게 됨  
비어있는 시퀀스를 넣어도 min() 함수에서 ValueError를 발생시킬 것이므로 따로 ValueError 예외를 발생시키지 않아도 됨

#### 중첩 루프
경우에 따라 1차원 이상을 반복해서 값을 찾아야 할 때 가장 쉽게 해결하기 위해서 중첩루프를 사용할 수 있음  
중첩루프는 값을 찾는 경우 break 키워드를 호출해야하는데 이런 경우 한 단계가 아니라 두 단계 이상을 벗어나야 하므로 정상적으로 동작하지 않음  
플래그나 예외를 사용하는 것은 좋은 방법이 아니므로 가장 좋은 방법은 가능하면 중첩을 풀어 1차원 루프로 만드는 것임

아래는 피해야 할 코드임

In [57]:
def search_nested_bad(array, desired_value):
    coords = None
    for i, row in enumerate(array):
        for j, cell in enumerate(row):
            if cell==desired_value:
                coords = (i, j)
                break
        if coords is not None:
            break
            
    if coords is None:
        raise ValueError(f"{desired_value} not found")
        
    logger.info("[%i, %i]에서 값 %r 찾음" % (*coords, desired_value))
    return coords

다음은 종료 플래그를 사용하지 않은 보다 간단하고 컴팩트한 형태의 코드

In [58]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell
            
def search_nested(array, desired_value):
    try:
        coord = next(
            coord 
            for (coord, cell) in _iterate_array2d(array) 
            if cell == desired_value
        )
    except StopIteration:
        raise ValueError(f"{desired_value} not found")
        
    logger.info("[%i, %i]에서 값 %r 찾음" % (*coord, desired_value))
    return coord

제너레이터를 통해 반복을 추상화 하였음.  
지금은 2차원 배열을 사용했으나 더 많은 차원의 배열을 사용하는 경우에도 클라이언트는 그것에 대해 알 필요 없이 기존 코드를 그대로 사용하면 됨  